### Import data
Dataset contains user rating data for movies collected from 1990s - 2000s.

movie ratings
movie meta data like genres and year
demographics of users age, zip code, gender identification, and occupation
An application for this kind of data could be recommendation systems

Import the 3 different files. In kaggle we need to adapt the orginal path to the data and specify the loading engine for read_table() to python to avoid warning messages.

In [22]:
import pandas as pd

In [23]:
pd.options.display.max_rows = 10

In [24]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('input/users.dat', sep='::',
 header=None, names=unames, engine='python')

In [15]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [25]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('input/ratings.dat', sep='::', header=None,
names=rnames, engine='python')

In [29]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('input/movies.dat', sep='::', header=None, 
                       names=mnames,encoding='latin-1', engine='python')

Now we can varify that the input data was loaded correcly, by having a look at the first rows of each file.

In [30]:
users.dtypes

user_id        int64
gender        object
age            int64
occupation     int64
zip           object
dtype: object

Note: Ages and occupation are encoded as integers.

In [31]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Movie genre is a pipe-separated string, so analysis of genre require some transformation to bring this column in a usable form.

In [32]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


We want to analyse the mean ratings for a specific movie by age and gender.

Next we need to merge all data into one dataframe to make handling easier.

Merging strategy is to first merge ratings with users and followed by merging movies.

Pandas infers which columns to use for the merge by finding overlapping names.

In [33]:
ratings.head(1)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760


In [34]:
movies.head(1)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy


In [35]:
data = pd.merge(pd.merge(ratings, users), movies)

Let's take a look at the merged data frame.names.

In [36]:
data[:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [37]:
data.iloc[0]

user_id                                            1
movie_id                                        1193
rating                                             5
timestamp                                  978300760
gender                                             F
age                                                1
occupation                                        10
zip                                            48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

## Calculate mean movie ratings for each film by gender using the pivot table method
The function pivot table reshapes the data frame depending on index and columns from input parameters.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

We create a new data frame with mean ratings for each move title and split gender as columns (F and M).

In [38]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')

In [39]:
mean_ratings[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


Filter movies with low rating (less than 250).

| group data by title and get size()

| extract list of titles with ratings by title >= 250

| use list to select mean ratings

In [41]:
ratings_by_title = data.groupby('title').size()
ratings_by_title[:10]

title
$1,000,000 Duck (1971)                37
'Night Mother (1986)                  70
'Til There Was You (1997)             52
'burbs, The (1989)                   303
...And Justice for All (1979)        199
1-900 (1994)                           2
10 Things I Hate About You (1999)    700
101 Dalmatians (1961)                565
101 Dalmatians (1996)                364
12 Angry Men (1957)                  616
dtype: int64

In [42]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles[:10]

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)'],
      dtype='object', name='title')

In [43]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings[:5]

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421


#### Show the top list of movies

In [44]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


In [45]:
top_male_ratings = mean_ratings.sort_values(by='M', ascending=False)
top_male_ratings[:10]

gender,F,M
title,,
"Godfather, The (1972)",4.314700,4.583333
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.481132,4.576628
"Shawshank Redemption, The (1994)",4.539075,4.560625
Raiders of the Lost Ark (1981),4.332168,4.520597
"Usual Suspects, The (1995)",4.513317,4.518248
Star Wars: Episode IV - A New Hope (1977),4.302937,4.495307
Schindler's List (1993),4.562602,4.491415
"Wrong Trousers, The (1993)",4.588235,4.478261
"Close Shave, A (1995)",4.644444,4.473795


## Measuring Rating Disagreement
Here we want to find the movies that are most different between male and female users.

In [46]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Sorting ascending my diff shows us the movies that are rated high by women, but low by men.

In [47]:
mean_ratings.sort_values(by='diff').head(10)

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Reversing the order shows us the movies that are rated how by men and low by woman.

In [48]:
mean_ratings.sort_values(by='diff', ascending=False).head(10)

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Finding movies with highest disagreement of viewers independent of gender:

>> calculate variance and standard deviation of the ratings

>> filter titles with ratings count equal or higher 250 (list active_titles from earlier)

>> sort movies in descending order

In [49]:
rating_std_by_title = data.groupby('title')['rating'].std()
rating_std_by_title[:5]

title
$1,000,000 Duck (1971)           1.092563
'Night Mother (1986)             1.118636
'Til There Was You (1997)        1.020159
'burbs, The (1989)               1.107760
...And Justice for All (1979)    0.878110
Name: rating, dtype: float64

In [50]:
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64